<a href="https://colab.research.google.com/github/SunshineGreeny/Dive-into-deep-learning-Pytorch/blob/main/chapter_convolutional-neural-networks/lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Networks (LeNet)



In [20]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
import torch.optim as optim

LeNet (LeNet-5) consists of two parts:
(i) a convolutional encoder consisting of two convolutional layers; and
(ii) a dense block consisting of three fully connected layers

In [9]:
import torch
from torch import nn
import torch.nn.functional as F

def init_cnn(m: nn.Module):
    """Initialize weights for CNNs."""
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        nn.init.xavier_uniform_(m.weight)

class LeNet(nn.Module):
    """The LeNet-5 model."""
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
            nn.Linear(120, 84), nn.Sigmoid(),
            nn.Linear(84, num_classes))

    def forward(self,X):
      return self.net(X)

    def layer_summary(self, X_shape):
        X = torch.randn(*X_shape)
        for layer in self.net:
            X = layer(X)
            print(layer.__class__.__name__, 'output shape:\t', X.shape)

Tools

In [21]:
class Trainer:
    def __init__(self, model, train_loader, test_loader, max_epochs, num_gpus, device="cuda"):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.max_epochs = max_epochs
        self.num_gpus = num_gpus
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.1) # Assuming lr is 0.1 based on LeNet init

    def fit(self):
        train_losses = []
        test_losses = [] # Although not used in the print statement, keeping for completeness

        for epoch in range(self.max_epochs):
            # 训练阶段
            self.model.train()
            total_loss, total_correct, total_samples = 0, 0, 0
            for X, y in self.train_loader:
                X, y = X.to(self.device), y.to(self.device)
                y_hat = self.model(X)
                loss = self.criterion(y_hat, y)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item() * y.size(0)
                total_correct += (y_hat.argmax(dim=1) == y).sum().item()
                total_samples += y.size(0)

            train_loss = total_loss / total_samples
            train_acc = total_correct / total_samples

            # 测试阶段
            self.model.eval()
            correct, total = 0, 0
            with torch.no_grad():
                for X, y in self.test_loader:
                    X, y = X.to(self.device), y.to(self.device)
                    y_hat = self.model(X)
                    correct += (y_hat.argmax(dim=1) == y).sum().item()
                    total += y.size(0)

            test_acc = correct / total

            print(f"Epoch {epoch+1}/{self.max_epochs}:"
                  f"Train Loss {train_loss:.4f}, Train acc {train_acc:.4f}, Test acc {test_acc:.4f}")

Inspect the model

In [10]:
def layer_summary(self, X_shape):
    X = torch.randn(*X_shape)
    for layer in self.net:
        X = layer(X)
        print(layer.__class__.__name__, 'output shape:\t', X.shape)

model = LeNet()
model.layer_summary((1, 1, 28, 28))

Conv2d output shape:	 torch.Size([1, 6, 28, 28])
Sigmoid output shape:	 torch.Size([1, 6, 28, 28])
AvgPool2d output shape:	 torch.Size([1, 6, 14, 14])
Conv2d output shape:	 torch.Size([1, 16, 10, 10])
Sigmoid output shape:	 torch.Size([1, 16, 10, 10])
AvgPool2d output shape:	 torch.Size([1, 16, 5, 5])
Flatten output shape:	 torch.Size([1, 400])
Linear output shape:	 torch.Size([1, 120])
Sigmoid output shape:	 torch.Size([1, 120])
Linear output shape:	 torch.Size([1, 84])
Sigmoid output shape:	 torch.Size([1, 84])
Linear output shape:	 torch.Size([1, 10])


Run an experiment to see how the LeNet-5 model fares on Fashion-MNIST

In [19]:
# Assuming data is already loaded and split into train and test loaders
# If not, you'll need to add code to load and prepare the data
data = FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(data, batch_size=128, shuffle=True)

data_test = FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
test_loader = DataLoader(data_test, batch_size=128, shuffle=False)


model = LeNet(lr=0.1)
model.apply(init_cnn) # Use apply instead of apply_init

trainer = Trainer(model=model, train_loader=train_loader, test_loader=test_loader, max_epochs=10, num_gpus=1)
trainer.fit()

100%|██████████| 26.4M/26.4M [00:00<00:00, 114MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 3.96MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 59.8MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 24.6MB/s]


Epoch 1/10:Train Loss 2.4492, Train acc 0.0968, Test acc 0.1000
Epoch 2/10:Train Loss 2.4197, Train acc 0.0986, Test acc 0.1000
Epoch 3/10:Train Loss 2.4351, Train acc 0.0982, Test acc 0.1000
Epoch 4/10:Train Loss 2.4284, Train acc 0.1005, Test acc 0.1000
Epoch 5/10:Train Loss 2.4232, Train acc 0.0999, Test acc 0.1000
Epoch 6/10:Train Loss 2.4258, Train acc 0.1002, Test acc 0.1000
Epoch 7/10:Train Loss 2.4365, Train acc 0.0983, Test acc 0.1000
Epoch 8/10:Train Loss 2.4278, Train acc 0.0988, Test acc 0.1000
Epoch 9/10:Train Loss 2.4177, Train acc 0.1001, Test acc 0.1000
Epoch 10/10:Train Loss 2.4136, Train acc 0.0984, Test acc 0.1000
